# nbcleanse CLI Interface

In [ ]:
#| default_exp cli

In [ ]:
#| export

from functools import reduce
from nbcleanse.notebook import Notebook

def get_nbs_paths(globs: list):
    ex_globs = [Path('.').glob(glob) for glob in globs]
    paths = list(reduce(lambda l1, l2: l1 + l2, ex_globs))
    return [p for p in paths if p.suffix == '.ipynb']

def report_paths(paths: list):
    print('🕵️  Notebooks Found:\n' + '\n'.join([str(p.absolute()) for p in paths]))

In [ ]:
#| export

from pathlib import Path
import sys
import contextlib
import os

def nbcleanse(**kwargs):
    "Clean all notebooks in the current directory using the given parameters"            
    quiet_context = contextlib.redirect_stdout(open(os.devnull, 'w')) if kwargs['quiet'] else contextlib.nullcontext()
    with quiet_context:
        paths = get_nbs_paths(kwargs['find_globs'])        
        report_paths(paths)
        for path in paths:                
            nb = Notebook(path)                
            if kwargs['clean_operations'] is not None:
                print(f'🧼 Cleaning {path} ...')   
                nb.clean(kwargs['clean_operations'])
                nb.save()
            # TODO: enable check mode
            # else:
            #     print(f'🧼 Checking {path} ...')
        print('✨ Done! ✨')

In [ ]:
#| export

from argparse import ArgumentParser
from nbcleanse.notebook import CleanOperations


parser = ArgumentParser(prog='nbcleanse')
parser.add_argument('-m', action='append_const', help='Clean cells and notebook-wide metadata', dest='clean_operations', const=CleanOperations.METADATA)
parser.add_argument('-o', action='append_const', help='Clean cells outputs', dest='clean_operations', const=CleanOperations.OUTPUTS)
parser.add_argument('-e', action='append_const', help='Clean cells execution counts', dest='clean_operations', const=CleanOperations.EXECUTION_COUNT)
parser.add_argument('-a', action='append_const', help='Clean EVERYTHING from a notebook', dest='clean_operations', const=CleanOperations.ALL)
# TODO: append const + argument that is the pattern to apply
parser.add_argument('-j', action=)
parser.add_argument('-q', action='store_true', help='Quiet mode', dest='quiet')
parser.add_argument(nargs='*', help='Glob patterns (relative to cwd) of notebooks to filter', dest='find_globs', default=['*.ipynb'])

def main():
    args = parser.parse_args()
    nbcleanse(**vars(args))

## export

In [1]:
from nbdev.export import nb_export

nb_export('cli.ipynb')